## 13 Основные принципы ООП  

## 13.1 Инкапсуляция

<dt><a href="https://ru.wikipedia.org/wiki/%D0%98%D0%BD%D0%BA%D0%B0%D0%BF%D1%81%D1%83%D0%BB%D1%8F%D1%86%D0%B8%D1%8F_%28%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5%29" title="Инкапсуляция (программирование)">Инкапсуляция</a><sup id="cite_ref-Pirs_5-0" class="reference"><a href="#cite_note-Pirs-5">[5]</a></sup>&nbsp;</dt>
<dd>Инкапсуляция&nbsp;— свойство системы, позволяющее объединить данные и методы, работающие с ними, в классе. Обычно инкапсуляцию отождествляют с сокрытием даных.</dd>
<dt>

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/8/82/CPT-OOP-interfaces.svg/1200px-CPT-OOP-interfaces.svg.png">

### Инкапсуляция: два смысла

#### Широкий смысл — объединение данных и методов

Инкапсуляция — это **упаковка данных и методов, работающих с ними, в единый объект (класс)**.

```
┌─────────────────────────────────────┐
│           BankAccount               │
├─────────────────────────────────────┤
│  Данные:                            │
│    - balance                        │
│    - owner                          │
│    - account_number                 │
├─────────────────────────────────────┤
│  Методы:                            │
│    - deposit()                      │
│    - withdraw()                     │
│    - get_balance()                  │
└─────────────────────────────────────┘

Всё, что связано с банковским счётом — в одном месте.
```

#### Узкий смысл — сокрытие реализации (data hiding)

Инкапсуляция — это **ограничение доступа к внутренним данным объекта**. Внешний код работает только через публичный интерфейс.

```
         Внешний код                         Объект
        ┌──────────┐                   ┌─────────────────┐
        │          │   get_balance()   │                 │
        │  Клиент  │ ───────────────▶  │  Публичные      │
        │          │   deposit(100)    │    методы       │
        │          │ ───────────────▶  │                 │
        └──────────┘                   ├─────────────────┤
              │                        │                 │
              │   __balance = 500      │  Приватные      │
              │ ─ ─ ─ ─ ─ ─ ─ ─ ─ ✗ ─▶ │    данные       │
              │   (прямой доступ)      │                 │
                                       └─────────────────┘
```

| Смысл | Суть | Пример в Python |
|-------|------|-----------------|
| Широкий | Группировка в класс | `class BankAccount: ...` |
| Узкий | Сокрытие данных | `self.__balance` (private) |

### Пример: банковский счёт

In [1]:
class BankAccount:
    def __init__(self, owner: str, initial_balance: float = 0):
        self.owner = owner              # Публичный атрибут
        self.__balance = initial_balance  # Приватный атрибут (__)
    
    def deposit(self, amount: float) -> float:
        """Публичный метод для пополнения."""
        if amount <= 0:
            raise ValueError("Сумма должна быть положительной")
        self.__balance += amount
        return self.__balance
    
    def withdraw(self, amount: float) -> float:
        """Публичный метод для снятия."""
        if amount <= 0:
            raise ValueError("Сумма должна быть положительной")
        if amount > self.__balance:
            raise ValueError("Недостаточно средств")
        self.__balance -= amount
        return self.__balance
    
    @property
    def balance(self) -> float:
        """Только чтение баланса (нельзя изменить напрямую)."""
        return self.__balance


# Использование
account = BankAccount("Иван Петров", 1000)

print(f"Владелец: {account.owner}")
print(f"Баланс: {account.balance}")

account.deposit(500)
print(f"После пополнения: {account.balance}")

account.withdraw(200)
print(f"После снятия: {account.balance}")

# Попытка прямого доступа к балансу
# account.__balance = 1000000  # ❌ AttributeError — нет такого атрибута
# account.balance = 1000000    # ❌ AttributeError — property только для чтения

Владелец: Иван Петров
Баланс: 1000
После пополнения: 1500
После снятия: 1300


### Уровни доступа в Python

Python использует **соглашения об именовании** (не строгие ограничения):

| Синтаксис | Уровень | Описание |
|-----------|---------|----------|
| `name` | Public | Доступен всем |
| `_name` | Protected | "Для внутреннего использования" (соглашение) |
| `__name` | Private | Name mangling → `_ClassName__name` |

> ⚠️ В Python нет **настоящей** приватности. `__name` можно обойти через `_ClassName__name`, но это считается плохой практикой.

In [ ]:
class Example:
    def __init__(self):
        self.public = "Я публичный"
        self._protected = "Я защищённый (соглашение)"
        self.__private = "Я приватный"

obj = Example()

print(obj.public)       # ✅ Работает
print(obj._protected)   # ✅ Работает (но не рекомендуется)
# print(obj.__private)  # ❌ AttributeError

# Name mangling — Python переименовывает __private в _Example__private
print(obj._Example__private)  # ✅ Работает (но это хак, не делайте так!)

# Посмотрим все атрибуты
print([attr for attr in dir(obj) if not attr.startswith('__')])

### Пример: учёт рабочего времени

In [16]:
class EmployeeTracker:
    def __init__(self, name: str, surname: str):
        self.name = name
        self.surname = surname
        self.__working_hours = 0
        
    def work(self, hours: int):
        if hours < 0:
            return
        self.__working_hours += hours
    
    def count_dayoff(self, hours: int):
        if hours < 0:
            return

        self.__working_hours = max(self.__working_hours - hours, 0)
    
    def reset(self):
        self.__working_hours = 0
    
    def get_total_working_hours(self):
        return self.__working_hours

petr_tracker = EmployeeTracker(name='Petr', surname='Petrov')
petr_tracker.work(hours=8)
petr_tracker.work(hours=4)
petr_tracker.count_dayoff(hours=20)
print(f'salary:', petr_tracker.get_total_working_hours() * 5000)

salary: 0


In [15]:
# Немного улучшим
from dataclasses import dataclass

@dataclass
class Employee:
    name: str
    surname: str


class EmployeeTracker:
    def __init__(self, employee: Employee):
        self.__employee = employee
        self.__working_hours = 0
        
    def work(self, hours: int):
        if hours < 0:
            return
        self.__working_hours += hours
    
    def count_dayoff(self, hours: int):
        if hours < 0:
            return
        self.__working_hours -= hours
    
    def reset(self):
        self.__working_hours = 0
    
    @property
    def working_hours(self) -> int:
        # light-weight method
        return self.__working_hours

    @property
    def employee_name(self) -> str:
        # light-weight method
        return f'{self.__employee.name} {self.__employee.surname}'

petr = Employee(name='Petr', surname='Petrov')
masha = Employee(name='Masha', surname='Petrova')

petr_tracker = EmployeeTracker(employee=petr)

petr_tracker.work(hours=8)
petr_tracker.work(hours=4)
petr_tracker.count_dayoff(hours=2)
print(f'Salary for {petr_tracker.employee_name} =', petr_tracker.working_hours * 5000)


Salary for Petr Petrov = 50000


## 13.2 Наследование

<dt><a href="https://ru.wikipedia.org/wiki/%D0%9D%D0%B0%D1%81%D0%BB%D0%B5%D0%B4%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5_(%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5)" title="Наследование (программирование)">Наследование</a><sup id="cite_ref-Pirs_5-1" class="reference"><a href="#cite_note-Pirs-5">[5]</a></sup>&nbsp;</dt>
<dd>Наследование&nbsp;— свойство системы, позволяющее описать новый класс на основе уже существующего с частично или полностью заимствующейся функциональностью. Класс, от которого производится наследование, называется базовым, родительским или суперклассом. Новый класс&nbsp;— потомком, наследником, дочерним или производным классом.</dd>
<dt>

<img src="https://files.realpython.com/media/ic-basic-inheritance.f8dc9ffee4d7.jpg">

In [17]:
class Animal:
    def __init__(self, name: str = 'Sharik'):
        self.name = name
    
    def eat(self):
        print("I can eat")


class Dog(Animal):
    def display(self):
        print(f"My name is {self.name}")

labrador = Dog()

labrador.name = "Kishmish"
labrador.eat()

labrador.display()

I can eat
My name is Kishmish


## Немного примеров

<img src="https://files.realpython.com/media/ic-initial-employee-inheritance.b5f1e65cb8d1.jpg">

In [15]:
class PayrollSystem:
    def calculate_payroll(self, employees):
        print("Calculating Payroll")
        print("===================")
        for employee in employees:
            print(f"Payroll for: {employee.id} - {employee.name}")
            print(f"- Check amount: {employee.calculate_payroll()}")
            print("")


class Employee:
    def __init__(self, _id, name):
        self.id = _id
        self.name = name

class SalaryEmployee(Employee):
    def __init__(self, _id, name, weekly_salary):
        super().__init__(_id, name)
        self.weekly_salary = weekly_salary

    def calculate_payroll(self):
        return self.weekly_salary * 4

class HourlyEmployee(Employee):
    def __init__(self, _id, name, hours_worked, hourly_rate):
        super().__init__(_id, name)
        self.hours_worked = hours_worked
        self.hourly_rate = hourly_rate

    def calculate_payroll(self):
        return self.hours_worked * self.hourly_rate

class CommissionEmployee(SalaryEmployee):
    def __init__(self, _id, name, weekly_salary, commission):
        super().__init__(_id, name, weekly_salary)
        self.commission = commission

    def calculate_payroll(self):
        fixed = super().calculate_payroll()
        return fixed + self.commission

    
salary_employee = SalaryEmployee(1, "John Smith", 1500)
hourly_employee = HourlyEmployee(2, "Jane Doe", 80, 15)
commission_employee = CommissionEmployee(3, "Kevin Bacon", 1000, 250)


# v1
payroll_system = PayrollSystem()
payroll_system.calculate_payroll(
    [salary_employee, hourly_employee, commission_employee]
)


Calculating Payroll
Payroll for: 1 - John Smith
- Check amount: 6000

Payroll for: 2 - Jane Doe
- Check amount: 1200

Payroll for: 3 - Kevin Bacon
- Check amount: 4250



## 13.3 Полиморфизм

<dt><a href="https://ru.wikipedia.org/wiki/%D0%9F%D0%BE%D0%BB%D0%B8%D0%BC%D0%BE%D1%80%D1%84%D0%B8%D0%B7%D0%BC_(%D0%B8%D0%BD%D1%84%D0%BE%D1%80%D0%BC%D0%B0%D1%82%D0%B8%D0%BA%D0%B0)">Полиморфизм</a><sup id="cite_ref-Pirs_5-2" class="reference"><a href="#cite_note-Pirs-5">[5]</a></sup>&nbsp;</dt>
<dd>Полиморфизм&nbsp;— свойство системы, позволяющее использовать объекты с одинаковым интерфейсом без информации о типе и внутренней структуре объекта. Другой вид полиморфизма&nbsp;— <a href="https://ru.wikipedia.org/wiki/%D0%9F%D0%B0%D1%80%D0%B0%D0%BC%D0%B5%D1%82%D1%80%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B8%D0%B9_%D0%BF%D0%BE%D0%BB%D0%B8%D0%BC%D0%BE%D1%80%D1%84%D0%B8%D0%B7%D0%BC" title="Параметрический полиморфизм">параметрический</a>&nbsp;— в ООП называют <a href="https://ru.wikipedia.org/wiki/%D0%9E%D0%B1%D0%BE%D0%B1%D1%89%D1%91%D0%BD%D0%BD%D0%BE%D0%B5_%D0%BF%D1%80%D0%BE%D0%B3%D1%80%D0%B0%D0%BC%D0%BC%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5" title="Обобщённое программирование">обобщённым программированием</a>.</dd>

<img src="https://javarush.com/images/article/1c3dd28a-1263-4db5-b8b3-06d172e3204c/original.png">

In [16]:
# Duck-typing

class Cat:
    def __init__(self, name, age):
        self.name = name
        self.age = age

    def info(self):
        print(f"I am a cat. My name is {self.name}. I am {self.age} years old.")

    def speak(self):
        print("Meow")

class Cow:
    def __init__(self, name, age):
        self.name = name
        self.age = age

    def info(self):
        print(f"I am a cow. My name is {self.name}. I am {self.age} years old.")

    def speak(self):
        print("Moo")


class Dog:
    def __init__(self, name, age):
        self.name = name
        self.age = age

    def info(self):
        print(f"I am a dog. My name is {self.name}. I am {self.age} years old.")

    def speak(self):
        print("Woof")


class Phone:
    def speak(self):
        print("Alloo")

    def info(self):
        print("I am phone - I'm animal")


cat1 = Cat("Marusya", 2.5)
dog1 = Dog("Gudron", 4)
cow1 = Cow("Dusia", 2)
phone = Phone()

for animal in (cat1, dog1, cow1, phone):
    animal.speak()
    animal.info()


Meow
I am a cat. My name is Marusya. I am 2.5 years old.
Woof
I am a dog. My name is Gudron. I am 4 years old.
Moo
I am a cow. My name is Dusia. I am 2 years old.
Alloo
I am phone - I'm animal


In [29]:
# Inheritance
import abc

class Animal(abc.ABC):
# class Animal(metaclass=abc.ABCMeta):
    def __init__(self, name, age):
        self.name = name
        self.age = age

    def info(self):
        print(f"I am a {self.__class__.__name__.lower()}. My name is {self.name}. I am {self.age} years old.")
    
    @abc.abstractmethod
    def speak(self):
        pass


class Cat(Animal):
    def speak(self):
        print("Meow")

        
class Cow(Animal):
    def speak(self):
        print("Moo")
    
    def eat(self):
        print('Eating')


class Dog(Animal):
    def speak(self):
        print("Woof")
        

class Elephant(Animal):
    def speak(self):
        print("UUuuuh")

        
cat1 = Cat("Marusya", 2.5)
dog1 = Dog("Gudron", 4)
cow1 = Cow("Dusia", 2)
elephant = Elephant('Big boss', 10)

animals: list[Animal] = [cat1, dog1, cow1, elephant]

for animal in animals:
    animal.speak()
    animal.info()


Meow
I am a cat. My name is Marusya. I am 2.5 years old.
Woof
I am a dog. My name is Gudron. I am 4 years old.
Moo
I am a cow. My name is Dusia. I am 2 years old.
UUuuuh
I am a elephant. My name is Big boss. I am 10 years old.


### Еще примеры

In [46]:
# Делаем вместе
# В рамках импортозамещения наш университет решил создать новый автомобиль "Маруся".
# Нужно создать информационную модель.

In [32]:
from dataclasses import dataclass
import abc


@dataclass
class Engine:
    power: int
    mark: str
    rotation: int


@dataclass
class Transmission:
    gears_count: int
    current_gear: int = 0

    
class MarusyaCar:
    def __init__(self, engine: Engine, transmission: Transmission, color: str = 'black', wheels_count: int = 4):
        self._color = color
        self.engine = engine
        self.wheels_count = wheels_count
        self.transmission = transmission
        self.__speed = 0
    
    def go(self, gear_increment: int = 1):
        if gear_increment < 0:
            return
    
        self.transmission.current_gear += gear_increment
        self.__speed += self.engine.power * 0.2 * self.transmission.current_gear
    
    @property
    def speed(self):
        return self.__speed
    
    @property
    def current_transmission_gear(self):
        return self.transmission.current_gear

    def replace_engine(self, engine: Engine):
        self.engine = engine
    
bmw_engine = Engine(power=200, mark='bmw', rotation=0)
toyota_engine = Engine(power=150, mark='toyota', rotation=0)
uaz_engine = Engine(power=1500, mark='uaz', rotation=0)

uaz_transmission = Transmission(gears_count=5)

marusya = MarusyaCar(engine=uaz_engine, transmission=uaz_transmission)
marusya.go()
marusya.go()
marusya.go()
print('toyota_engine', marusya.speed, marusya.current_transmission_gear)
marusya.replace_engine(bmw_engine)
marusya.go()
marusya.go()
print('bmw_engine', marusya.speed, marusya.current_transmission_gear)

toyota_engine 1800.0 3
bmw_engine 2160.0 5
